In [1]:
import numpy as np

data = np.genfromtxt("data/bio_train.csv", delimiter=",")[:,3:]
labels_true = np.genfromtxt("data/bio_train.csv", delimiter=",")[:,0]


In [3]:
from Coreset import *

coreset = Coreset(data)
# print(coreset.X[0])
# print(coreset.mu)
# print(coreset.X[0] - coreset.mu)
coreset.get_q()
#print(np.linalg.norm(coreset.X))
#print(coreset.q[:10])
print(sum(coreset.q))

cs, rnd_indices = coreset.get_coreset(70000)
print(cs.shape)



145750it [00:00, 160871.69it/s]
0.9999999999999949
(70000, 74)
227821


In [20]:
from itertools import product

eps = delta = np.arange(0.01, 1, 0.1)
 
for e, d in product(eps, delta):
    print(f'epsilon: {e:.2f}, delta: {d:.2f}, m: {coreset.get_m(eps=e, delta=d)}')
print(f'epsilon: 1, delta: 1, m: {coreset.get_m(eps=1, delta=1)}')


epsilon: 0.01, delta: 0.01, m: 569592233
epsilon: 0.01, delta: 0.11, m: 569568254
epsilon: 0.01, delta: 0.21, m: 569561787
epsilon: 0.01, delta: 0.31, m: 569557893
epsilon: 0.01, delta: 0.41, m: 569555097
epsilon: 0.01, delta: 0.51, m: 569552914
epsilon: 0.01, delta: 0.61, m: 569551124
epsilon: 0.01, delta: 0.71, m: 569549606
epsilon: 0.01, delta: 0.81, m: 569548288
epsilon: 0.01, delta: 0.91, m: 569547124
epsilon: 0.11, delta: 0.01, m: 4707373
epsilon: 0.11, delta: 0.11, m: 4707175
epsilon: 0.11, delta: 0.21, m: 4707122
epsilon: 0.11, delta: 0.31, m: 4707090
epsilon: 0.11, delta: 0.41, m: 4707066
epsilon: 0.11, delta: 0.51, m: 4707048
epsilon: 0.11, delta: 0.61, m: 4707034
epsilon: 0.11, delta: 0.71, m: 4707021
epsilon: 0.11, delta: 0.81, m: 4707010
epsilon: 0.11, delta: 0.91, m: 4707001
epsilon: 0.21, delta: 0.01, m: 1291592
epsilon: 0.21, delta: 0.11, m: 1291537
epsilon: 0.21, delta: 0.21, m: 1291523
epsilon: 0.21, delta: 0.31, m: 1291514
epsilon: 0.21, delta: 0.41, m: 1291508
epsil

## Epsilon and Delta Values chosen from the parameter search

epsilon, delta
(0.6, 0.5)
(0.7, 0.2)
(0.8, 0.7)
(0.9, 0.1)
(1, 1)


In [21]:
params = [
(0.6, 0.5),
(0.7, 0.2),
(0.8, 0.7),
(0.9, 0.1),
(1, 1)]

for eps, delta in params:
    print(f'epsilon: {eps:.2f}, delta: {delta:.2f}, m: {coreset.get_m(eps=eps, delta=delta)}')

epsilon: 0.60, delta: 0.50, m: 158209
epsilon: 0.70, delta: 0.20, m: 116237
epsilon: 0.80, delta: 0.70, m: 88992
epsilon: 0.90, delta: 0.10, m: 70317
epsilon: 1.00, delta: 1.00, m: 56954
